Estación: bogt 

Día: 011 año 2015 

Gpsweek: 1827 según el calendario https://www.ngs.noaa.gov/CORS/Gpscal.shtml


%cat igs18270.sp3

**Datos en formato sp3** <a href="http://www.gpstk.org/doxygen/classgpstk_1_1SP3Data.html">link</a>

clk: The clock bias or drift for P|V (microsec|1).

sig [4]: Four-vector of integer exponents for estimated sigma of position,clock or velocity,clock rate; sigma = base**n units are mm,psec or 10^-4 mm/sec,psec/sec); base in head

sdev [4]:std dev of 3 positions (XYZ,mm) and clock (psec) or velocities(10^-4 mm/sec) and clock rate (10^-4 ps/s) 

correlation [6]: elements of correlation matrix: xy,xz,xc,yz,yc,zc 

orbitPredFlag: orbit prediction flag, 'P' in file 

x [3]: The three-vector for position | velocity (m | dm/s). 


<a href="http://www.gpstk.org/pythondoc/search.html?q=sp3&check_keywords=yes&area=default">métodos de este objeto</a>

SP32NAV --> http://www.gpstk.org/bin/view/Documentation/UsingSP3toRinexNav
http://folcomuns.firabcn.es/Globalgeo_Proceedings/Proceedings/Geodesia%20y%20Navegaci%C3%B3n/GNSS%20Products/Phase-based%20GNSS%20data%20processing....pdf

[method for method in dir(object) if callable(getattr(object, method))]

In [36]:
import gpstk
import numpy as np
import pandas as pd

In [37]:
header, data = gpstk.readSP3("data/igs18270.sp3") #generador
print "Agencia: ",header.agency, "\nCoordenadas: ",header.coordSystem , "\nNumero de epocas: ",header.numberOfEpochs, "\nSistema (tiempo): ",header.timeSystem,"\nIntervalos (min): ", header.epochInterval/60  # 15 minutos

Agencia:   IGS 
Coordenadas:  IGb08 
Numero de epocas:  96 
Sistema (tiempo):  GPS 
Intervalos (min):  15.0


In [38]:
store = gpstk.SP3EphemerisStore()
store.loadSP3File("data/igs18270.sp3")
type(store) #http://www.gpstk.org/doxygen/classgpstk_1_1SP3EphemerisStore.html

gpstk.gpstk.SP3EphemerisStore

In [39]:
sats = store.getSatList()
print type(sats)
print [str(sat_id) for sat_id in sats]

<type 'tuple'>
['GPS 1', 'GPS 2', 'GPS 3', 'GPS 4', 'GPS 5', 'GPS 6', 'GPS 7', 'GPS 8', 'GPS 9', 'GPS 10', 'GPS 11', 'GPS 12', 'GPS 13', 'GPS 14', 'GPS 15', 'GPS 16', 'GPS 17', 'GPS 18', 'GPS 19', 'GPS 20', 'GPS 21', 'GPS 22', 'GPS 23', 'GPS 24', 'GPS 25', 'GPS 27', 'GPS 28', 'GPS 29', 'GPS 30', 'GPS 31', 'GPS 32']


In [40]:
time0 = store.getInitialTime()
print type(time0)

<class 'gpstk.gpstk.CommonTime'>


In [41]:
store.getClockInitialTime()

<gpstk.gpstk.CommonTime; proxy of <Swig Object of type 'std::map< gpstk::CommonTime,gpstk::ObsEpoch >::key_type *' at 0x7f389876fcc0> >

In [42]:
#print store.getXvt(sats[-1],store.getClockInitialTime()) #why you no work!

In [43]:
#store.ndata()#Get number of SP3 files in FileStore #ejemplos de store
#print store
#store.getInterpolationOrder()

# Con Pandas

In [44]:
a = {"sat":[],"clk":[],"x":[],"y":[],"z":[],"sod":[],"sdev":[],"sig":[],"orbitPred":[],"corr":[]}

In [45]:
for datum in data:
    #print datum.sat, type(datum.time), datum.x, datum.sig, gpstk.YDSTime(datum.time).sod
    a["sat"].append(str(datum.sat))
    a["clk"].append(datum.clk)
    a["x"].append(datum.x[0])
    a["y"].append(datum.x[1])
    a["z"].append(datum.x[2])
    a["sod"].append(gpstk.YDSTime(datum.time).sod)
    a["sdev"].append(datum.sdev)
    a["sig"].append(datum.sig)
    a["orbitPred"].append(datum.orbitPredFlag)
    a["corr"].append(datum.correlation)

df = pd.DataFrame(a)

#compare with .nav
nheader,ndata=gpstk.readRinex3Nav("bogt0110.15n")
bcestore = gpstk.GPSEphemerisStore() 

for ndato in ndata:
    print ndato.sat
    break
    """
    ephem = ndato.toGPSEphemeris()
    bcestore.addEphemeris(ephem)
bcestore.SearchNear() """

In [46]:
df.head()

,clk,corr,orbitPred,sat,sdev,sig,sod,x,y,z
0,4.594811e-322,"[0, 0, 0, 0, 0, 0]",False,GPS -1,"[0, 0, 0, 0]","[0, 0, 0, 0]",0.0,7.905050e-323,3.162020e-322,9.881313e-324
1,-1.031158e+01,"[0, 0, 0, 0, 0, 0]",False,GPS 1,"[0, 0, 0, 0]","[8, 9, 10, 136]",0.0,-2.258700e+04,-1.373325e+04,-3.441579e+03
2,5.411187e+02,"[0, 0, 0, 0, 0, 0]",False,GPS 2,"[0, 0, 0, 0]","[8, 6, 9, 131]",0.0,3.602560e+03,1.761547e+04,2.002291e+04
3,1.398603e+02,"[0, 0, 0, 0, 0, 0]",False,GPS 3,"[0, 0, 0, 0]","[8, 10, 8, 111]",0.0,-1.300038e+04,-1.594684e+04,1.677691e+04
4,-5.477739e+00,"[0, 0, 0, 0, 0, 0]",False,GPS 4,"[0, 0, 0, 0]","[8, 9, 9, 138]",0.0,-1.568542e+04,-1.761835e+04,-1.277747e+04


In [58]:
print len(np.unique(df.sod)) #96 epocas!
print 24*3600/900 # intervalos de 15 minutos en un día.. se pueden interpolar

96
96
